In [1]:
import math

import pandas as pd
import re
import requests
import csv

from time import sleep

from requests.exceptions import HTTPError
from bs4 import BeautifulSoup

In [2]:
#Função para requisição da página:
def request_page(URL):
    # Agentes de usuários de acesso:
    my_user_agents = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    try:
      site = requests.get(URL, headers=my_user_agents)
      site.raise_for_status()
    except HTTPError as erro:
      print(erro)#caso apresente algum erro
    else:
      return site.text
    sleep(30)
    
    

In [3]:
# Dicionário para armazenar as informações do site:
dados = {'produto':[], 'preco':[], 'open_box':[], 'prime_ninja':[]}

#Produto para busca na web site:
produto = input('Produto para buscar: ')
if produto == '':
    produto = 'mouse'
produto = produto.replace(' ','-')

#Número da página:
numero_pagina = 1

Produto para buscar: gabinete


In [4]:
#URL de busca:
URL=f'https://www.kabum.com.br/busca/{produto}?page_number={numero_pagina}&page_size=20&facet_filters=&sort=most_searched'



In [5]:
# Usuário de agente web:
site = request_page(URL)

# Leitura da página:
pagina = BeautifulSoup(site, 'html.parser')


In [6]:
# Pegando a quantidades de produtos disponível na consulta:
quantidade_de_produtos = pagina.find('div', attrs={'id':'listingCount'})
quantidade_de_produtos = quantidade_de_produtos.get_text().strip()
quantidade_de_produtos = re.findall(r'\b\d+\b',quantidade_de_produtos)[0]

# Pegando a quantidade de páginas para consulta:
numero_pagina = math.ceil(int(quantidade_de_produtos)/20)
print(f'Número de páginas: {numero_pagina}')

Número de páginas: 114


In [7]:
for num in range(1, numero_pagina+1):
    print(f'Página: {num}')
    url = f'https://www.kabum.com.br/busca/{produto}?page_number={num}&page_size=20&facet_filters=&sort=most_searched'
    paginas = request_page(url)
    site_kabum = BeautifulSoup(paginas, 'html.parser')
    #Loop para coletar as informações de cada página:
    for produtos in site_kabum.find_all('div', attrs={'class':'productCard'}):
        sleep(0.5)
        #Descrição do produto (nome, marca, cor, tamanho)
        descrição = produtos.find('span', attrs={'class':re.compile('nameCard')}).get_text().strip()
        dados['produto'].append(descrição)
        
        #preço
        preco = produtos.find('span', attrs={'class':'priceCard'}).get_text().strip()
        preco = re.sub("\$|\.|\-", "", str(preco))
        preco = preco.replace('R','').replace(',','.').strip()
        dados['preco'].append(preco)
        
        #se o produto é do tipo open box, ou seja, se já foi aberto da embalagem
        #e recondicionado pela Kabum
        open_box = produtos.find('div', attrs={'class':'openboxTagCard'})
        if open_box == None:
            open_box = 0
        else:
            open_box = 1
        dados['open_box'].append(open_box) 
        
        
        #Se o produto apresenta os desconto e vantagem da Prime Ninja,
        #ou seja, plano de assinatura da kabum:        
        ninja = produtos.find('div', attrs={'class':'primeLogoTagCard'})
        if ninja == None:
            ninja = 0
        else:
            ninja = 1
        dados['prime_ninja'].append(ninja)
            
    
    

Página: 1
Página: 2
Página: 3
Página: 4
Página: 5
Página: 6
Página: 7
Página: 8
Página: 9
Página: 10
Página: 11
Página: 12
Página: 13
Página: 14
Página: 15
Página: 16
Página: 17
Página: 18
Página: 19
Página: 20
Página: 21
Página: 22
Página: 23
Página: 24
Página: 25
Página: 26
Página: 27
Página: 28
Página: 29
Página: 30
Página: 31
Página: 32
Página: 33
Página: 34
Página: 35
Página: 36
Página: 37
Página: 38
Página: 39
Página: 40
Página: 41
Página: 42
Página: 43
Página: 44
Página: 45
Página: 46
Página: 47
Página: 48
Página: 49
Página: 50
Página: 51
Página: 52
Página: 53
Página: 54
Página: 55
Página: 56
Página: 57
Página: 58
Página: 59
Página: 60
Página: 61
Página: 62
Página: 63
Página: 64
Página: 65
Página: 66
Página: 67
Página: 68
Página: 69
Página: 70
Página: 71
Página: 72
Página: 73
Página: 74
Página: 75
Página: 76
Página: 77
Página: 78
Página: 79
Página: 80
Página: 81
Página: 82
Página: 83
Página: 84
Página: 85
Página: 86
Página: 87
Página: 88
Página: 89
Página: 90
Página: 91
Página: 

In [8]:
try:
    df = pd.DataFrame(dados)
    df.to_csv(path_or_buf=f'{produto}.csv',sep=';',index=False, encoding='latin-1')
except Exception  as erro:
    print(erro)
else:
    print('Arquivo carregado em um CSV')


'latin-1' codec can't encode character '\u2013' in position 15: ordinal not in range(256)
